# Project 1, Part 5, Data Visualization



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

# import additonal items for gmaps
import gmaps

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# 1.5 Example of a Data Visualization created using Python

The data science team would like for you to create an example of a data visualization using Python from data in a Pandas dataframe containing data from an SQL query.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  Any query of your choosing.  You can write a query from scratch.  You can use a query from a previous problem in this project.  You can use a query from the labs.  The idea is to come up with a query whose resulting data will make for an excellent quality data visualization. 

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.  Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.

Once you have the data in a Pandas dataframe, you may write as much Python code and use as many code cells as you wish to produce the data visualization.

You may only use Python modules that are currently installed in the Anaconda Docker container.  You may not install additional modules or any other software.

All work must be done in Docker in your VM in AWS.  You may not use any external data visualization systems, such as Tableau, etc.

You may use any code from the labs to pattern your code after, however for the data visualization, you cannot wholesale copy a data visualization from the labs.

Ensure that it is properly titled, including titles for axes if present.

Ensure that when you check this Juptyer Notebook into GitHub that the data visualization is clearly visible.  

If you want to use Google Maps for your data visualization, this is fine, however, the image will not show up in GitHub.  So, just save the image to an image file, include it in the repo, and add a markdown cell to display the image file. Also with Google Maps, do NOT check gmap_api_key.txt into GitHub for security reasons.  If the grader needs to run it, they will supply their own gmap_api_key.txt.


In [4]:
rollback_before_flag = True
rollback_after_flag = True

# for all sales by product in berkeley zipcodes
query = """

select  cu.zip as customer_zip,
        cu.city as customer_city,
        z.latitude,
        z.longitude,
        count(cu.customer_id) as total_customers_by_zip,
        sum(l.quantity) as total_meals_purchased

from stores as s
    join customers as cu
        on s.store_id = cu.closest_store_id
    join sales as sa
        on cu.customer_id = sa.customer_id
    join line_items as l
        on sa.store_id = l.store_id and sa.sale_id = l.sale_id
    join products as p
        on l.product_id = p.product_id
    join zip_codes as z
         on cu.zip = z.zip

where s.city = 'Berkeley'
group by customer_zip, customer_city, z.latitude, z.longitude
order by total_meals_purchased desc

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,customer_zip,customer_city,latitude,longitude,total_customers_by_zip,total_meals_purchased
0,94608,Emeryville,37.8361,-122.2862,51122,76178
1,94602,Oakland,37.8041,-122.2070,47159,70691
2,94530,El Cerrito,37.9216,-122.2984,45910,68941
3,94606,Oakland,37.7918,-122.2450,41003,61573
4,94611,Oakland,37.8301,-122.2023,41163,61135
...,...,...,...,...,...,...
139,94588,Pleasanton,37.7375,-121.8818,107,153
140,94565,Pittsburg,38.0158,-121.9070,93,148
141,94957,Ross,37.9631,-122.5637,98,146
142,94516,Canyon,37.8339,-122.1650,101,140


In [5]:
# Use as many code cells as you need to create and display your data visualization
# Load gmap info
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

In [6]:
# lat long for berkeley store
sather_gate_berkeley = (37.870260430419115, -122.25950168579497)

# Mark the store
markers = gmaps.marker_layer([sather_gate_berkeley])

# Weighted heatmap based on total meals purchased
heatmap_layer = gmaps.heatmap_layer(df[['latitude', 'longitude']], weights=df['total_meals_purchased'])
heatmap_layer.max_intensity = 60000
heatmap_layer.point_radius = 20

In [7]:
# Display figure

# Title: Berkeley Store Sales by Customer Zip Code
# Marker: Store site
# Heatmap: total meals purchased by customer zip code

fig = gmaps.figure(center=sather_gate_berkeley, map_type='HYBRID', zoom_level=10)

fig.add_layer(markers)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))